In [7]:
import pickle

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, roc_auc_score

In [8]:
df = pd.read_csv("http://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [9]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
target = "Survived"
ignore_cols = ["Name", "PassengerId", "Ticket", "Cabin"]
numeric_preds = list(set(df._get_numeric_data().columns) - set([target] +ignore_cols))
cat_preds = list(set(df.columns) - set([target] + ignore_cols + numeric_preds))

In [12]:
df_prep = pd.concat([df[numeric_preds], df[[target]], pd.get_dummies(df[cat_preds], drop_first=True)], axis=1).fillna(0)
df_prep.head()

,Age,Parch,SibSp,Fare,Pclass,Survived,Sex_male,Embarked_Q,Embarked_S
0,22.0,0,1,7.2500,3,0,1,0,1
1,38.0,0,1,71.2833,1,1,0,0,0
2,26.0,0,0,7.9250,3,1,0,0,1
3,35.0,0,1,53.1000,1,1,0,0,1
4,35.0,0,0,8.0500,3,0,1,0,1


In [13]:
predictors = list(set(df_prep.columns) - set([target]))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_prep[predictors], df_prep[target], test_size=0.2, random_state=42)

In [16]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True, max_depth=5, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, oob_score=True, random_state=42)

In [17]:
pd.DataFrame(confusion_matrix(y_test, rf.predict(X_test)))

,0,1
0,95,10
1,22,52


In [18]:
roc_auc_score(y_test, rf.predict(X_test))

0.8037323037323038

In [20]:
pickle.dump(rf, open("./model.pkl", "wb"))